# **Analysis of Launch Site Locations Using Folium**

The success rate of rocket launches may be influenced by various factors, including payload mass, orbit type, and other operational considerations. Additionally, the geographical location and proximity of a launch site—specifically, the initial position of rocket trajectories—may also play a crucial role. The identification of an optimal site for rocket launches necessitates the evaluation of multiple factors, some of which may be discerned through an analysis of existing launch site locations.

In previous exploratory data analysis labs, the SpaceX launch dataset was visualised using `matplotlib` and `seaborn`, revealing preliminary correlations between launch site locations and success rates. In this lab, a more interactive approach to visual analytics will be undertaken using `Folium`.

## Objectives

This lab comprises the following tasks:

* **TASK 1:** Mark all launch sites on a map  
* **TASK 2:** Indicate successful and failed launches for each site on the map  
* **TASK 3:** Compute the distances between each launch site and its surrounding locations  

Upon completion of these tasks, geographical patterns associated with launch sites should become more evident.

First, the required Python packages for this lab will be imported:



In [1]:
# Geospatial visualization library
import folium

# Library for web scrapping
# import wget # Only required if want to work with coursera IBM dataset
#from bs4 import BeautifulSoup

# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Standard libraries for handling dates, file operations, and serialization
import datetime
import os
import pickle

# SpaceX-specific module (custom or third-party)
import spacex

# Importing directory paths from SpaceX config
from spacex.config import RAW_DATA_DIR, INTERIM_DATA_DIR, PROCESSED_DATA_DIR, FIGURES_DIR

# Display settings for pandas DataFrame
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [2]:
from folium.plugins import MarkerCluster  # Import folium MarkerCluster plugin
from folium.plugins import MousePosition  # Import folium MousePosition plugin
from folium.features import DivIcon       # Import folium DivIcon plugin

## Task 1: Marking All Launch Sites on a Map

The first step involves plotting the locations of all launch sites on a map using their respective latitude and longitude coordinates.


In [3]:
spacex_pkl_file = os.path.join(INTERIM_DATA_DIR, 'dataset_part_2.pkl')
spacex_df=pd.read_pickle(spacex_pkl_file)

Now, the coordinates for each launch site can be examined.



In [4]:
# Select relevant sub-columns: `Launch_Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch_Site', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['Launch_Site'], as_index=False, observed=False).first()
launch_sites_df = launch_sites_df[['Launch_Site', 'Latitude', 'Longitude']]
launch_sites_df

,Launch_Site,Latitude,Longitude
0,CCSFS SLC-40,28.561857,-80.577366
1,KSC LC-39A,28.608058,-80.603956
2,VAFB SLC-4E,34.632093,-120.610829


The coordinates listed above are merely numerical values and do not provide intuitive insights regarding the locations of the launch sites. While those with strong geographical knowledge may interpret these coordinates mentally, a visual representation will be more effective for most. 

To enhance understanding, these locations will be visualised by pinning them on a map.

First, a `folium.Map` object must be created, with its initial centre set to the **NASA Johnson Space Center** in Houston, Texas.


In [5]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

A highlighted circle with a text label can be added to a specific coordinate using `folium.Circle`, allowing for better visual representation of the location. For example:



In [6]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(
    nasa_coordinate, 
    radius=1000, 
    color='#000ed3', 
    fill=True
)

# Create a marker with a text label
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#ff0000;"><b>%s</b></div>' % 'NASA JSC',
    )
)

# Create a marker with a space-related icon and custom colour
NASA_marker = folium.Marker(
    location=nasa_coordinate,
    icon=folium.Icon(icon="space-shuttle",  
                     prefix="fa", 
                     color="black"),  
    popup="Welcome to the Dark Side"
)

In [7]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
# Add elements to the map
site_map.add_child(circle)
site_map.add_child(marker)
site_map.add_child(NASA_marker)  # Fixed variable name

A small blue circle should now be visible near the city of Houston. By zooming in, a larger circle can be observed for better visibility.

Next, a circle will be added for each launch site in the `Launch_Sites` DataFrame.


In [8]:
ls_circles=[]
ls_markers=[]
ls_icons=[]
USA_avg_coord=[31,-100]
#print(USA_avg_coord)
#print(launch_sites_df)

# For each launch site, add a Circle object based on its coordinate (Latitude, Longitude) values. 
# In addition, add Launch site name as a popup label
for i in range(len(launch_sites_df)):
    ls_coord=[launch_sites_df['Latitude'][i],launch_sites_df['Longitude'][i]]
    ls_circles.append(folium.Circle(ls_coord, 
                                    radius=100, 
                                    color='#d35400', 
                                    fill=True)
                     )
    ls_icons.append(folium.map.Marker(ls_coord,
                                      icon=folium.Icon(icon="shuttle-space", 
                                                       prefix="fa", 
                                                       color="darkblue")
                                                        ).add_child(folium.Popup(launch_sites_df['Launch_Site'][i])
                                     )
                    )
    ls_markers.append(folium.map.Marker(ls_coord,
                                        icon=DivIcon(icon_size=(50,50),
                                                     icon_anchor=(0,0),
                                                     html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_sites_df['Launch_Site'][i],
                                                     )
                                        )
                      )

In [9]:
# Initial the map
site_map = folium.Map(
    location=USA_avg_coord, 
    zoom_start=4)

site_map.add_child(ls_circles[0])
site_map.add_child(ls_markers[0])
site_map.add_child(ls_icons[0])                                      
site_map.add_child(ls_circles[1])
site_map.add_child(ls_markers[1])
site_map.add_child(ls_icons[1]) 
site_map.add_child(ls_circles[2])
site_map.add_child(ls_markers[2])
site_map.add_child(ls_icons[2]) 
#site_map.add_child(ls_circles[3])
#site_map.add_child(ls_markers[3])

* **Are all launch sites situated near the Equator?**  
  No, not all launch sites are located precisely on the Equator. However, they are positioned as far south as possible within U.S. territory. This strategic placement maximises the rocket’s velocity by taking advantage of the Earth's rotational speed, which is greatest at the Equator. The centrifugal force generated by Earth's rotation helps reduce the fuel required for launch while increasing the velocity necessary for reaching orbit.  

* **Are all launch sites in close proximity to the coast?**  
  Yes, most launch sites are located near the coast. This ensures that rockets are launched over open water, minimising risks to populated areas in the event of a failure. Alternatively, launch sites may also be situated in remote desert regions for the same reason—reducing potential hazards to inhabited areas.



# Task 2: Marking Successful and Failed Launches for Each Site on the Map  

The next step involves enhancing the map by incorporating launch outcomes for each site. This will allow for an assessment of which sites have higher success rates.  

It is important to recall that the `spacex_df` DataFrame contains detailed launch records, with the `class` column indicating whether a launch was successful or not.



In [10]:
spacex_df.tail(10)

,Launch_Site,Latitude,Longitude,Class
111,CCSFS SLC-40,28.561857,-80.577366,1
112,CCSFS SLC-40,28.561857,-80.577366,1
113,KSC LC-39A,28.608058,-80.603956,1
114,CCSFS SLC-40,28.561857,-80.577366,1
115,KSC LC-39A,28.608058,-80.603956,1
116,CCSFS SLC-40,28.561857,-80.577366,1
117,KSC LC-39A,28.608058,-80.603956,1
118,CCSFS SLC-40,28.561857,-80.577366,1
119,KSC LC-39A,28.608058,-80.603956,1
120,CCSFS SLC-40,28.561857,-80.577366,1


Next, markers will be created for all launch records.  

* A **green marker** will be used for successful launches (`'Class' == 1`).  
* A **red marker** will be used for failed launches (`'Class' == 0`).  

Since each launch occurs at one of the four designated launch sites, multiple launch records will share the same coordinates. To avoid clutter and improve readability, a **Marker Cluster** will be used to group markers at the same location.

First, a `MarkerCluster` object will be created.



In [11]:
marker_cluster = MarkerCluster()


A new column will be created in `Launch_Sites` dataframe called `marker_color` to store the marker colors based on the `Class` value. A function will be applied to check the values in the `Class` column:  

* If `Class = 1`, the `marker_color` value will be set to green.  
* If `Class = 0`, the `marker_color` value will be set to red.  

In [12]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['Class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch_Site,Latitude,Longitude,Class,marker_color
111,CCSFS SLC-40,28.561857,-80.577366,1,green
112,CCSFS SLC-40,28.561857,-80.577366,1,green
113,KSC LC-39A,28.608058,-80.603956,1,green
114,CCSFS SLC-40,28.561857,-80.577366,1,green
115,KSC LC-39A,28.608058,-80.603956,1,green
116,CCSFS SLC-40,28.561857,-80.577366,1,green
117,KSC LC-39A,28.608058,-80.603956,1,green
118,CCSFS SLC-40,28.561857,-80.577366,1,green
119,KSC LC-39A,28.608058,-80.603956,1,green
120,CCSFS SLC-40,28.561857,-80.577366,1,green


In [13]:
# Process each launch record in the dataset
for index, record in spacex_df.iterrows():
    # Generate a marker at the launch coordinates
    # The marker's icon is adjusted to reflect the launch outcome:
    # - Green for successful launches
    # - Red for failed launches
    marker = folium.Marker(
        [float(record['Latitude']), float(record['Longitude'])],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    
    # Incorporate the marker into the cluster for improved visual clarity
    marker_cluster.add_child(marker)

# Integrate the marker cluster into the existing site map
site_map.add_child(marker_cluster)



By analysing the colour-coded markers within the marker clusters, the relative success rates of each launch site can be easily identified. Sites with a higher concentration of green markers indicate a greater number of successful launches, while those with more red markers suggest a lower success rate.



# TASK 3: Calculating the Distances Between a Launch Site and Its Nearby Locations  

The next step involves analysing the proximities of launch sites to key locations such as railways, highways, and bodies of water. Understanding these distances provides insights into site accessibility, safety considerations, and logistical factors.  

To facilitate this analysis, a `MousePosition` feature will be added to the map. This tool allows for real-time retrieval of coordinates by hovering over any point of interest (e.g., a railway or a road), enabling precise identification of relevant locations.


In [14]:
# Create a new map
site_map = folium.Map(
    location=USA_avg_coord, 
    zoom_start=4)

# Add Mouse Position to get the coordinate (Latitude, Longitude) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topleft',
    separator='   Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:  ',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map.add_child(ls_circles[0])
site_map.add_child(ls_markers[0])
site_map.add_child(ls_icons[0])                                      
site_map.add_child(ls_circles[1])
site_map.add_child(ls_markers[1])
site_map.add_child(ls_icons[1]) 
site_map.add_child(ls_circles[2])
site_map.add_child(ls_markers[2])
site_map.add_child(ls_icons[2]) 

Now, zoom in on a launch site and examine its proximity to key infrastructure such as railways, highways, and coastlines. By moving the mouse over these features, their coordinates can be obtained from the display in the top-left corner of the map. These coordinates should be recorded to facilitate distance calculations relative to the launch site.

The distance between two points on the map can be determined using their `Latitude` and `Longitude` values with the following method:

In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [16]:
dic_positions={'position': ['Railway station', 'Coastline'] ,'Latitude':[28.57695, 28.56201],'Longitude':[-80.58431, -80.5678]}
positions = pd.DataFrame.from_dict(dic_positions)
display(positions)

,position,Latitude,Longitude
0,Railway station,28.57695,-80.58431
1,Coastline,28.56201,-80.56780


In [17]:
launch_sites_df

,Launch_Site,Latitude,Longitude
0,CCSFS SLC-40,28.561857,-80.577366
1,KSC LC-39A,28.608058,-80.603956
2,VAFB SLC-4E,34.632093,-120.610829


In [18]:
# find coordinate of the closet coastline and railway station
distance_railstation = calculate_distance(launch_sites_df.iloc[0]['Latitude'], launch_sites_df.iloc[0]['Longitude'], positions.iloc[0]['Latitude'], positions.iloc[0]['Longitude'])
distance_coastline   = calculate_distance(launch_sites_df.iloc[0]['Latitude'], launch_sites_df.iloc[0]['Longitude'], positions.iloc[1]['Latitude'], positions.iloc[1]['Longitude'])
print(
    f"The distance from the launch site {launch_sites_df.iloc[0]['Launch_Site']} "
    f"to the nearest {positions.iloc[0]['position']} is {distance_railstation:.2f} km.\n"
    f"The distance from the launch site {launch_sites_df.iloc[0]['Launch_Site']} "
    f"to the nearest {positions.iloc[1]['position']} is {distance_coastline:.2f} km."
)

The distance from the launch site CCSFS SLC-40 to the nearest Railway station is 1.81 km.
The distance from the launch site CCSFS SLC-40 to the nearest Coastline is 0.93 km.


A railway marker is added at the **nearest railway station** and **coastline** using an **icon**, along with a **distance label** indicating its proximity to the **CCSFS SLC-40** launch site. A blue **line** is drawn to visually connect the railway station and the  launch site, highlighting their spatial relationship.


In [19]:
# Define railway coordinates
railway_coords = [positions.iloc[0]['Latitude'], positions.iloc[0]['Longitude']]

# Add a railway marker with an icon instead of a circle
railway_marker = folium.Marker(
    location=railway_coords,
    icon=folium.Icon(icon="train", prefix="fa", color="blue"),
    popup="Nearest Railway Station"
)

# Add a distance label near the railway station
railway_distance_marker = folium.Marker(
    location=railway_coords,
    icon=DivIcon(
        icon_size=(50, 30),
        icon_anchor=(0, 0),
        html=f"<div style='text-align: center; font-size: 12px; color:#d35400; font-weight: bold;'>"
             f"Railway<br>{distance_railstation:.2f} km</div>"
    )
)

# Create a PolyLine to connect the railway station and the launch site
railway_lines = folium.PolyLine(
    locations=[railway_coords, [launch_sites_df.iloc[0]['Latitude'], launch_sites_df.iloc[0]['Longitude']]], 
    color="blue", 
    weight=2, 
    opacity=0.7
)



In [20]:
# Define coastline coordinates
coastline_coords = [positions.iloc[1]['Latitude'], positions.iloc[1]['Longitude']]

# Add a railway marker with an icon instead of a circle
coast_marker = folium.Marker(
    location=coastline_coords,
    icon=folium.Icon(icon="anchor", prefix="fa", color="red"),
    popup="Nearest coastline"
)

# Add a distance label near the railway station
coastline_distance_marker = folium.Marker(
    location=coastline_coords,
    icon=DivIcon(
        icon_size=(50, 30),
        icon_anchor=(0, 0),
        html=f"<div style='text-align: center; font-size: 12px; color:#d35400; font-weight: bold;'>"
             f"Coastline<br>{distance_coastline:.2f} km</div>"
    )
)

# Create a PolyLine to connect the railway station and the launch site
coastline_lines = folium.PolyLine(
    locations=[coastline_coords, [launch_sites_df.iloc[0]['Latitude'], launch_sites_df.iloc[0]['Longitude']]], 
    color="red", 
    weight=2, 
    opacity=0.7
)

In [21]:
# Create a new map
site_map = folium.Map(
    location=USA_avg_coord, 
    zoom_start=4)

# Add coordinates
site_map.add_child(mouse_position)

# Add NASA site
site_map.add_child(NASA_marker)

# Add railway station to the map
site_map.add_child(railway_marker)
site_map.add_child(railway_distance_marker)
site_map.add_child(railway_lines)

# Add coastline elements to the map
site_map.add_child(coastline_lines)
site_map.add_child(coastline_distance_marker)
site_map.add_child(coast_marker)

# Add launch sites
site_map.add_child(ls_circles[0])
site_map.add_child(ls_markers[0])
site_map.add_child(ls_icons[0])                                      
site_map.add_child(ls_circles[1])
site_map.add_child(ls_markers[1])
site_map.add_child(ls_icons[1]) 
site_map.add_child(ls_circles[2])
site_map.add_child(ls_markers[2])
site_map.add_child(ls_icons[2]) 



Copyright © 2021 IBM Corporation. All rights reserved.
